<a href="https://colab.research.google.com/github/azuredge1101/deep-learn/blob/main/CBE109012%E6%9C%9F%E6%9C%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_dir = Path('/content/drive/MyDrive/archive/Fish_Dataset/Fish_Dataset')

# **創造一個資料夾**

In [ ]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# Sample 200 images from each class
samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(200, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
image_df

,Filepath,Label
0,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
1,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
2,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
3,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
4,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
...,...,...
195,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
196,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
197,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish
198,/content/drive/MyDrive/_LABELED-FISHES-IN-THE-...,Positive_fish


In [ ]:
train_df, test_df = train_test_split(image_df, train_size = 0.7, shuffle = True, random_state = 1)

**載入圖片**

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
                                                                 validation_split = 0.2)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input)

In [ ]:
train_images = train_generator.flow_from_dataframe(dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training')

val_images = train_generator.flow_from_dataframe(dataframe = train_df,
                                                x_col = 'Filepath',
                                                y_col = 'Label',
                                                target_size = (224, 224),
                                                color_mode = 'rgb',
                                                class_mode = 'categorical',
                                                batch_size = 32,
                                                shuffle = True,
                                                seed = 42,
                                                subset = 'validation')
test_images =  test_generator.flow_from_dataframe(dataframe = test_df,
                                                 x_col = 'Filepath',
                                                 y_col = 'Label',
                                                 target_size = (224,224),
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 batch_size = 32,
                                                 shuffle = False)

Found 1008 validated image filenames belonging to 9 classes.
Found 252 validated image filenames belonging to 9 classes.
Found 540 validated image filenames belonging to 9 classes.


**載入模型**

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
input_shape = (224, 224, 3),
include_top = False,
weights = 'imagenet',
pooling = 'avg')

pretrained_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(9, activation = 'softmax')(x)


model =tf.keras.Model(inputs=inputs, outputs = outputs)


model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(train_images,
                  validation_data = val_images,
                  epochs = 10,
                  callbacks = [
                      tf.keras.callbacks.EarlyStopping(
                      monitor = 'val_loss', 
                      patience = 3,
                      restore_best_weights= True)
                  ])

Epoch 1/10
32/32 [==============================] - 267s 8s/step - loss: 0.7953 - accuracy: 0.7798 - val_loss: 0.1188 - val_accuracy: 0.9603
Epoch 2/10
32/32 [==============================] - 49s 2s/step - loss: 0.0559 - accuracy: 0.9861 - val_loss: 0.0683 - val_accuracy: 0.9881
Epoch 3/10
32/32 [==============================] - 52s 2s/step - loss: 0.0217 - accuracy: 0.9980 - val_loss: 0.0589 - val_accuracy: 0.9802
Epoch 4/10
32/32 [==============================] - 49s 2s/step - loss: 0.0132 - accuracy: 0.9990 - val_loss: 0.0436 - val_accuracy: 0.9841
Epoch 5/10
32/32 [==============================] - 49s 2s/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.0411 - val_accuracy: 0.9881
Epoch 6/10
32/32 [==============================] - 49s 2s/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0478 - val_accuracy: 0.9802
Epoch 7/10
32/32 [==============================] - 49s 2s/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0297 - val_accuracy: 0.9841
Epoch 8/10
32/32 [=

In [ ]:
results = model.evaluate(test_images, verbose = 0)

print("Test Loss {:.5f}".format(results[0]))
print("Test Accuracy {:.2f}%".format(results[1] * 100))

Test Loss 0.01833
Test Accuracy 99.63%
